<a href="https://colab.research.google.com/github/dlbkv/AI-Step/blob/master/module7/lab23_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [2]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200140 entries, 0 to 200139
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   blog_id  200140 non-null  int64  
 1   userId   200140 non-null  int64  
 2   ratings  200140 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 4.6 MB


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [8]:
try:
  from surprise import Dataset, Reader
except ModuleNotFoundError:
  !pip install -q surprise

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userId", "blog_id", "ratings"]], reader)

In [9]:

from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [10]:
from surprise import BaselineOnly, SVD, KNNBasic, CoClustering, accuracy

In [11]:
df['userId'].describe()

count    200140.000000
mean       2545.710158
std        1446.195478
min          10.000000
25%        1314.000000
50%        2552.000000
75%        3795.000000
max        5010.000000
Name: userId, dtype: float64

In [12]:
df['blog_id'].describe()

count    200140.000000
mean       5652.533621
std        2970.685946
min           1.000000
25%        2906.000000
50%        5994.000000
75%        8510.000000
max        9755.000000
Name: blog_id, dtype: float64

In [15]:
try:
  import optuna
except ModuleNotFoundError:
  !pip install -q optuna
import time

def objective(trial):

    start_time = time.time()
    print(f"Iteration: {trial.number}")


    algo_type = trial.suggest_categorical('algo_type', ['BaselineOnly','SVD','KNNBasic', 'CoClustering'])

    if algo_type == 'BaselineOnly':
        method_type = trial.suggest_categorical('method_type', ['sgd', 'als'])
        if method_type == 'sgd':
          params = {
              'method': 'sgd',
              'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1.0, log=True),
              'n_epochs': 20,
          }
        else:
          params = {
              'method': 'als',
              'reg_u': trial.suggest_float('reg_u', 1e-6, 1.0, log=True),
              'reg_i': trial.suggest_float('reg_i', 1e-6, 1.0, log=True),
              'n_epochs': 20,
          }

        algo = BaselineOnly(bsl_options=params)

    elif algo_type == 'SVD':
        params = {
            'n_factors': trial.suggest_int('n_factors', 10, 150),
            'reg_all': trial.suggest_float('reg_all', 1e-6, 1.0, log=True),
            'lr_all': trial.suggest_float('lr_all', 1e-6, 1.0, log=True),

        }

        algo = SVD(**params)

    elif algo_type == 'KNNBasic':

        params = {
            'k': 10,
            'min_k': 1,
        }

        sim_params = {
            'min_support': 1,
            'name': trial.suggest_categorical('name', ['cosine', 'pearson', 'msd']),
            'user_based': trial.suggest_categorical('user_based', [True, False]),
        }

        algo = KNNBasic(**params, sim_options=sim_params)

    elif algo_type == 'CoClustering':
      params = {
            'n_epochs': 20,
            'n_cltr_u': trial.suggest_int('n_cltr_u', 1, 20),
            'n_cltr_i': trial.suggest_int('n_cltr_i', 1, 20),

        }

      algo = CoClustering(**params)

    algo.fit(trainset)

    preds = algo.test(testset)
    metric = accuracy.mse(preds, verbose=False)

    end_time = time.time()
    iteration_time = end_time - start_time

    print(f"mae = {accuracy.mae(preds, verbose=False)}")
    print(f"mse = {accuracy.mse(preds, verbose=False)}")
    print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
    print(f"fcp = {accuracy.fcp(preds, verbose=False)}")
    print()
    print(f"Iteration time: {iteration_time} seconds")

    return metric

In [16]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

Iteration: 0
Estimating biases using als...
mae = 1.6377463410467856
mse = 3.4295575906643427
rmse= 1.8519064745997145
fcp = 0.4980994931285224

Iteration time: 2.9758400917053223 seconds
Iteration: 1
Computing the pearson similarity matrix...
Done computing similarity matrix.
mae = 1.7089734227548363
mse = 4.051388439477693
rmse= 2.0128061107512796
fcp = 0.5173040703549466

Iteration time: 6.170613050460815 seconds
Iteration: 2
mae = 1.7136347261444633
mse = 3.993492496245612
rmse= 1.9983724618412886
fcp = 0.5035558329660974

Iteration time: 8.61327338218689 seconds
Iteration: 3
mae = 1.6123458273160138
mse = 3.1477087285046474
rmse= 1.7741783248886362
fcp = 0.4975021375646615

Iteration time: 3.7211077213287354 seconds
Iteration: 4
Estimating biases using sgd...
mae = 1.646795643764697
mse = 3.517425719306911
rmse= 1.8754801303418043
fcp = 0.5000720104415141

Iteration time: 2.186018943786621 seconds
Iteration: 5
Estimating biases using sgd...
mae = 1.6117872886603035
mse = 3.1214951

In [17]:
study.best_value

3.121478177711594

In [18]:
best_params = study.best_params
best_params

{'algo_type': 'BaselineOnly',
 'method_type': 'sgd',
 'learning_rate': 1.0001582373161025e-06}